# Dynamic Models for Building Energy Management
## 1st of June

Léa Brette
Antoine Guédon
Adrian Heinrich-Phan
Florian Bouniard

## Schema of our building



!["Building"](Bat.png)

## Dimensions of the building

Here, we compute the dimensions of our 2 rooms:
we have chosen to take into account only one wall in contact with the rest of the chalet and one wall with the 2m² window in contact with the outside

In [1]:
l_br=3 #lenght of the bedroom
L_br=4 #width of the bedroom
l_d= 3 #lenght of the dressing
L_d= 2 #width of the dressing
height= 3   #height under ceiling

S_br= l_br*L_br  #total surface bedroom
S_d= l_d*L_d    #total surface dressing

S_glass= 2   # surface of the glass taken into account in the study

S_wall_3=l_d*height  # surface of the wall between the dressing and the bedroom
S_wall_4=L_d*height  # surface of the wall of the dressing
S_wall_2= L_br*height # surface de droite 
S_wall_1= l_br*height - S_glass #surface du haut
 



h_in = 8 #convection coefficient for inside, W/m² K;
h_out= 25 #convection coefficient for outside, W/m² K;
T_out = 10   #temperature outside, °C;
T_in = 20   #temperature inside, °C;
T_i0 = T_out +273 #temp out, K;


σ = σ = 5.67e-8     # W/(m²⋅K⁴) Stefan-Bolzmann constant


## Desciption of the building


### Thermo-physical properties

In [2]:
#Sapin
wood = {'Conductivity': 0.06,
            'Density': 450,
            'Specific heat': 2300,
            'Width':0.2 ,
            'Surface': S_wall_1+S_wall_2+S_wall_3+S_wall_4
             }

whool = {'Conductivity': 0.034,
              'Density': 40, 
              'Specific heat': 1030,
              'Width':0.25 ,
              'Surface': S_wall_1+S_wall_2+S_wall_3+S_wall_4 }

glass = {'Conductivity': 1 ,
         'Density': 2500,
         'Specific heat': 0.8,
         'Width':0.04 ,
         'Surface': S_glass}

air = {'Conductivity': 0.024 ,
         'Density': 1.293,
         'Specific heat': 1005,
         'Width':0.1 ,
         'Surface': S_glass }   
          

### Radiative properties

In [3]:
ε_wLW = 0.85    # long wave emmisivity: wall surface (concrete)
ε_gLW = 0.90    # long wave emmisivity: glass pyrex
α_wood = 0.65    # short wave absortivity: white smooth surface
α_glass = 0.38    # short wave absortivity: reflective blue glass
τ_gSW = 0.30    # short wave transmitance: reflective blue glass

irradiance = 1000   #W/m2

Phi0 = S_wall_1*irradiance*α_wood  #wood
Phia = S_glass*irradiance*α_glass  #glass

# Thermal circuit of our building
![](Thermal.png)


## Thermal conductances

### Conduction

In [4]:
G_1 = wood['Conductivity'] / wood['Width'] * S_wall_1
G_2 = wood['Conductivity'] / wood['Width'] * S_wall_1
G_3 = whool['Conductivity'] / whool['Width'] * S_wall_1
G_4 = whool['Conductivity'] / whool['Width'] * S_wall_1
G_10= wood['Conductivity'] / wood['Width'] * S_wall_2
G_17= wood['Conductivity'] / wood['Width'] * S_wall_3
G_14= wood['Conductivity'] / wood['Width'] * S_wall_4
G_13= glass['Conductivity'] / glass['Width'] * S_glass


### Convection

In [5]:
G_0 = h_out * S_wall_1
G_5= h_in * S_wall_1
G_11= h_in * S_wall_2
G_16= h_in * S_wall_3
G_15= h_in * S_wall_4
G_6 = h_in * S_wall_3
G_9 = h_in * S_glass
G_12= h_out * S_glass


### Long wave radiation

In [6]:
#Long Wave radiation
Tm = T_in + 273   # K, mean temp for radiative exchange
Fwg = glass['Surface']  / wood['Surface']    # view factor wall-glass

#Between window and wall_3
G7_1 = 4 * σ * Tm**3 * ε_wLW / (1 - ε_wLW) * S_wall_3
G7_12 = 4 * σ * Tm**3 * Fwg * S_wall_3 
G7_2 = 4 * σ * Tm**3 * ε_gLW / (1 - ε_gLW) * glass['Surface']
G_7 = 1 / (1 / G7_1 + 1 / G7_12 + 1 / G7_2) 

#Between wall_1 and wall_3
G8_1 = 4 * σ * Tm**3 * ε_wLW / (1 - ε_wLW) * S_wall_3
G8_12 = 4 * σ * Tm**3 * Fwg * S_wall_3 
G8_2 = 4 * σ * Tm**3 * ε_gLW / (1 - ε_gLW) * S_wall_1 
G_8 = 1 / (1 / G8_1 + 1 / G8_12 + 1 / G8_2)

## Thermal capacities

In [7]:
C_1 = wood['Density'] * wood['Specific heat'] * wood['Surface'] * wood['Width']      #wood capacity wall 1
C_2 = whool['Density'] * whool['Specific heat'] * whool['Surface'] * whool['Width']  #whool capacity wall 1

## System of Algebraic Differential Equations (DAE)

### Incidence matrix A

In [8]:
import numpy as np
nq = 18                         # n° of branches
n0 = 13                         # n° of nodes
A = np.zeros([nq,n0])             # n° of branches X n° of nodes
A[0, 0] = 1                     # branch 0: -> node 0
A[1, 0], A[1, 1] = -1, 1        # branch 1: node 0 -> node 1
A[2, 1], A[2, 2] = -1, 1        # branch 2: node 1 -> node 2
A[3, 2], A[3, 3] = -1, 1        # branch 3: node 2 -> node 3
A[4, 3], A[4, 4] = -1, 1        # branch 4: node 3 -> node 4
A[5, 4], A[5, 5] = -1, 1        # branch 5: node 4 -> node 5
A[6, 5], A[6, 6] =  1,-1        # branch 6: node 6 -> node 5
A[7, 6], A[7, 7] =  1,-1        # branch 7: node 7 -> node 6
A[8, 4], A[8, 6] = -1, 1        # branch 8: node 4 -> node 6
A[9, 5], A[9, 7] =  1,-1        # branch 9: node 7 -> node 5
A[10, 8] = 1                    # branch 10: -> node 8
A[11, 5], A[11, 8] = 1, -1      # branch 11: node 8 -> node 5
A[12, 9] = 1                    # branch 12: -> node 9
A[13, 7], A[13, 9] = 1, -1      # branch 13: node 9 -> node 7
A[14, 12] = 1                   # branch 14: -> node 12
A[15, 11], A[15, 12] = 1, -1    # branch 15: node 12 -> node 11  
A[16, 10], A[16, 11] = 1, -1    # branch 16: node 11 -> node 10
A[17, 6], A[17, 10] = 1, -1     # branch 17: node 10 -> node 6
print(A)

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0. -1.  0.  0.]]


### Conductance matrix G

Photo matrice G

In [9]:
G = np.diag(np.hstack([G_0,G_1,G_2,G_3,G_4,G_5,G_6,G_7,G_8,G_9,G_10,G_11,G_12,G_13,G_14,G_15,G_16,G_17]))
print(G)

[[175.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.        ]
 [  0.           2.1          0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.        ]
 [  0.           0.           2.1          0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.        ]
 [  0.           0.           0.           0.952        0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.        ]
 [  0.           0.           0.           0.           0.952
    0.           0.           0.           0.           0.
   

### Capacity Matrix C

Photo matrice C

In [10]:
C = np.diag(np.hstack([0,C_1,0,C_2,0,0,0,0,0,0,0,0,0]))

### Temperature vector Teta

In [11]:
Teta = np.zeros(n0)

### Temperature source vector b

In [12]:
b = np.zeros(nq)
b[0] = T_out
b[10] = T_in
b[12] = T_out
b[14] = T_in

### Heat flow source vector f

In [13]:
f = np.zeros(n0)
f[0] = Phi0
f[9] = Phia

### Heat flow vector q

In [14]:
q = np.zeros(nq)

### Computation of Teta

In [15]:
Teta = np.linalg.inv(A.T @ G @ A) @ (A.T @ G @ b + f)
print(f'θ = {Teta} °C')

θ = [35.97756621 34.1080841  32.23860198 28.11474438 23.99088677 23.92544542
 23.90277978 24.64969748 23.78356185 24.92484874 22.3980936  22.34166787
 22.25702927] °C


### Computation of q

In [16]:
q = G @ (-A @ Teta+b)
print(f'q = {q} W')

q = [-4.54607409e+03  3.92591244e+00  3.92591244e+00  3.92591244e+00
  3.92591244e+00  3.66471569e+00 -1.63192612e+00  2.16952981e+00
  2.61196749e-01  1.15880331e+01 -1.36208227e+01 -1.36208227e+01
 -7.46242437e+02  1.37575629e+01 -4.06265268e+00 -4.06265268e+00
 -4.06265268e+00 -4.06265268e+00] W


### Computation of y

In [17]:
y = np.zeros(n0)         # nodes
y[[5]] = Teta[[5]]              # nodes (temperatures) of interest
print(f'y = ', y)

y =  [ 0.          0.          0.          0.          0.         23.92544542
  0.          0.          0.          0.          0.          0.
  0.        ]


# State space representation

### Matrices As, Bs, Cs and Ds for state space representation

In [18]:
import dm4bem
[As, Bs, Cs, Ds] = dm4bem.tc2ss(A, G, b, C, f, y)
print('As = \n', As, '\n')
print('Bs = \n', Bs, '\n')
print('Cs = \n', Cs, '\n')
print('Ds = \n', Ds, '\n')

As = 
 [[-3.87914846e-07  9.30727297e-08]
 [ 1.87049078e-06 -4.39069436e-06]] 

Bs = 
 [[2.94842116e-07 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.68481209e-09 0.00000000e+00]
 [0.00000000e+00 5.74966462e-07 1.77698483e-06 1.68252288e-07
  0.00000000e+00 3.55396965e-08]] 

Cs = 
 [[0.         0.05802889]] 

Ds = 
 [[0.         0.2152981  0.66433766 0.06233536 0.         0.01328675]] 



### Input vector u

In [19]:
bT = np.array([T_out, T_in, T_out, T_in])     #the four temperature sources
fQ = np.array([Phi0, Phia])         # [Φo, Φi, Qa, Φa]
u = np.hstack([bT, fQ])
print(f'u = {u}')

u = [  10.   20.   10.   20. 4550.  760.]


### Steady-state value of the output of the state-space representation yss

In [20]:
yss = (-Cs @ np.linalg.inv(As) @ Bs + Ds) @ u
print(f'yss = {yss} °C')


yss = [23.92544542] °C


### Computation of the error between the steady-state values obtained from the system of DAE θ[[5]] and the output of the state-space representation yss

In [21]:
print(f'Max error between DAE and state-space: \
{max(abs(Teta[5] - yss)):.2e} °C')

Max error between DAE and state-space: 4.62e-14 °C


# Simulate step reponse

### Time step

We need to find a step response that satisfies Euler's stability conditions:

So we need to have: 
$\Delta t < \frac{-2}{min(\lambda_i)}$ with $\lambda_i$ the eigenvalues of As

In [22]:
λ = np.linalg.eig(As)[0]    # eigenvalues of matrix As

print('Time constants: \n', -1 / λ, 's \n')
print('2 x Time constants: \n', -2 / λ, 's \n')
dtmax = 2 * min(-1. / λ)
print(f'Maximum time step: {dtmax:.2f} s = {dtmax / 60:.2f} min')

Time constants: 
 [2899519.55267877  225544.01033887] s 

2 x Time constants: 
 [5799039.10535753  451088.02067775] s 

Maximum time step: 451088.02 s = 7518.13 min


We have therefore obtained a maximum time step value of: ECRIRE LA VLAUER OBTENUE

For greater safety, we choose a lower time step such as:

In [23]:
# time step arrondie à la min inférieure
dt = np.floor(dtmax / 60) * 60   # s
print(f'dt = {dt} s = {dt / 60:.0f} min')

dt = 451080.0 s = 7518 min


### Settling time

Settling time represent the time our system takes to reach and stay within a range of 5% of the final value.

$T_s= 4*max(\frac{-1}{\lambda_i})$



In [24]:
# settling time
time_const = np.array([int(x) for x in sorted(-1 / λ)])
print('4 * Time constants: \n', 4 * time_const, 's \n')

t_settle = 4 * max(-1 / λ)
print(f'Settling time: \
{t_settle:.0f} s = \
{t_settle / 60:.1f} min = \
{t_settle / (3600):.2f} h = \
{t_settle / (3600 * 24):.2f} days')

4 * Time constants: 
 [  902176 11598076] s 

Settling time: 11598078 s = 193301.3 min = 3221.69 h = 134.24 days


### Step reponse
**Duration**  
  
The duration of the simulation needs to be larger than the estimated settling time. This requires a corresponding number of time steps in the time vector.  

We choose to stop the program one hour later than the setteling time value to be sure of reaching the desired temperature.

In [25]:

duration = np.ceil(t_settle / 3600) * 3600
n = int(np.floor(duration / dt))    # number of time steps
t = np.arange(0, n * dt, dt)        # time vector for n time steps

print(f'Duration = {duration} s')
print(f'Number of time steps = {n}')

Duration = 11599200.0 s
Number of time steps = 25


### Input vector
b vector represent the temperatures sources
$$ b=
\left(\begin{array}{cc} 
T_{out} \\0\\....\\ T_{in} \\ T_{out}\\0\\0\\ T_{in} \\0\\0\\0\\0\\0
\end{array}\right)
$$
So we have b_T the nonzero elements of the b vector:
$$
b_T=
\left(\begin{array}{cc} 
T_{out} \\ T_{in} \\ T_{out} \\ T_{in} 
\end{array}\right)
$$ 
And $f_Q$ the nonzero elements of f the vector of flow sources :
$$
f_Q=
\left(\begin{array}{cc} 
\Phi _0 \\ \Phi _0
\end{array}\right)
$$
With the expression of $b_T$ and $f_Q$ we can compute the input vector u:

$$
u=
\left(\begin{array}{cc} 
T_{out} \\ T_{in} \\ T_{out} \\ T_{in} \\ \Phi _0 \\ \Phi _0
\end{array}\right)
=
\left(\begin{array}{cc} 
T_{out}(0) & ... & T_{out}(n-1) \\ T_{in}(0) & ... & T_{in}(n-1)  \\ T_{out}(0) & ... & T_{out}(n-1) \\ T_{in}(0) & ... & T_{in}(n-1) \\ \Phi _0(0) &...& \Phi_0(n-1) \\ \Phi _0(0) &...& \Phi_0(n-1)
\end{array}\right)

In [ ]:
u = np.array([T_out,T_in,T_out,T_out,Phi0,Phi0])

### Time integration